In [ ]:
# %% Calculus 2 - Section 11.75
#    Code challenge: curve (arc) length

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [1]:
import numpy             as np
import sympy             as sym
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import scipy.integrate   as spi
import math
import mpmath

from scipy.signal                     import find_peaks
from IPython.display                  import display,Math
from google.colab                     import files
from IPython.display                  import Audio
from scipy.io                         import wavfile
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


In [ ]:
# %% Exercise 0a
#    Sympy, or how often times exact solutions fail

# Parametric function and derivatives
t  = sym.symbols('t')

xt = sym.cos(2*t)
dx = sym.diff(xt)

yt = sym.sin(t**2)
dy = sym.diff(yt)

# Plot
p = sym.plot_parametric(xt,yt,(t,0,sym.pi),title='A parametric function',ylabel='y(t)',xlabel='x(t)',legend=True,label='function',show=False)
p._backend.process_series()
lines = p._backend.ax.get_lines()
lines[0].set_linewidth(2)

p._backend.fig.savefig('fig16_codechallenge_75_exercise_0a.png')
files.download('fig16_codechallenge_75_exercise_0a.png')

# Curve class (C)
C = sym.Curve((xt,yt),(t,0,sym.pi))

# Integrand
integrand = sym.sqrt(dx**2 + dy**2)

# Intermediate reults
display(Math('dx = %s' %sym.latex(dx))), print()
display(Math('dy = %s' %sym.latex(dy))), print()
display(Math('%s' %sym.latex(sym.Integral(sym.sqrt( 1+(dx/dy)**2 )*dx,(t,0,sym.pi)))))

# Antiderivative (will crash because out of RAM)
# integral = sym.integrate(integrand,(t,0,sym.pi))


In [ ]:
# %% Exercise 0b
#    Numpy, or how empirical solutions are better (but more code and less
#    accuracy)

# Parametric function
t = np.linspace(0,np.pi,151)
x = np.cos(2*t)
y = np.sin(t**2)

# Approximation (loop)
seg_lengths = np.zeros(len(t))

for i in range(1,len(t)):

    # Deltas
    dx = x[i] - x[i-1]
    dy = y[i] - y[i-1]

    # Segment length
    seg_lengths[i] = np.sqrt(dx**2 + dy**2)

# Approximation (no loop)
dx = x[1:] - x[:-1]
dy = y[1:] - y[:-1]
seg_lengths = np.append(0,np.sqrt(dx**2 + dy**2))

# Plotting
cmap  = plt.cm.plasma(np.linspace(.1,.9,len(t)))

phi = (1 + np.sqrt(5)) / 3
_,axs = plt.subplots(1,3,figsize=(3*phi*5,5))
axs[0].scatter(x,y,s=20,c=cmap)
axs[0].set(xlabel='x(t)',ylabel='y(t)',title=r'$\bf{A}$)  The curve')

for ti in np.linspace(len(t)*.2,len(t)*.95,4,dtype=int):
    axs[0].annotate(f't={t[ti]:.1f}',[x[ti],y[ti]])
    axs[1].axvline(t[ti],linestyle=':',color=cmap[ti,:],zorder=-4)
    axs[2].axvline(t[ti],linestyle=':',color=cmap[ti,:],zorder=-4)

axs[1].scatter(t,seg_lengths,s=20,c=cmap)
axs[1].set(xlabel='t',ylabel='length (a.u.)',title=r'$\bf{B}$)  Segment lengths')

axs[2].scatter(t,np.cumsum(seg_lengths),s=20,c=cmap)
axs[2].set(xlabel='t',ylabel='length (a.u.)',title=r'$\bf{C}$)  Cumulative lengths')

print(f'Total curve length for n = {len(t)}: {np.sum(seg_lengths)}'), print()

plt.tight_layout()

plt.savefig('fig17_codechallenge_75_exercise_0b.png')
plt.show()
files.download('fig17_codechallenge_75_exercise_0b.png')


In [ ]:
# %% Exercise 0c
#    spi.simpson, or how empirical solutions are better (ideal for empirical
#    data with known values)

# Parameters
a = 0
b = np.pi
t = np.linspace(a,b,501)

# Derivative (empirical, np.gradient() is an alternative to np.diff())
dx_dt = np.gradient(np.cos(2*t),t)
dy_dt = np.gradient(np.sin(t**2),t)

# Simpson's rule to integrate
integrand    = np.sqrt(dx_dt**2 + dy_dt**2)
curve_length = spi.simpson(integrand,x=t)

print(f'Arc length spi.simpson: {curve_length}')
print(f'Arc length using numpy: {np.sum(seg_lengths)}')


In [ ]:
# %% Exercise 0d
#    spi.quad, or an half way solution (great for functions with known
#    derivatives)

# Parametric function and derivatives
p   = sym.symbols('p')

xp  = sym.cos(2*p)
dxp = sym.diff(xp)

yp  = sym.sin(p**2)
dyp = sym.diff(yp)

# Integrand
integrand = sym.lambdify(p,sym.sqrt(dxp**2 + dyp**2))

# Integrate
a   = 0
b   = np.pi
l,_ = spi.quad(integrand,a,b)

print(f'Arc length using spi.quad: {l}')
